**Imports**

In [1]:
!pip install tensornetwork

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.3/364.3 kB 4.4 MB/s eta 0:00:00


In [24]:
import numpy as np
import matplotlib.pyplot as plt
import tensornetwork as tn

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

**Simulation** **Parameters**

In [4]:
# Grid Parameters
N_QUBITS = 4
GRID_POINTS = 2**N_QUBITS

In [5]:
# Physical Domain Parameters
X_MIN, X_MAX = 0.0, 1.0
x_grid = np.linspace(X_MIN, X_MAX, GRID_POINTS)

In [6]:
# Fluid and time Parameters
VISCOSITY = 0.01
DT = 0.001
N_STEPS = 100

In [7]:
# More Simulation Parameters
DX = (X_MAX - X_MIN) / (GRID_POINTS - 1)
MAX_BOND_DIMENSION = 16

**CREATE** **INITIAL** **STATE** **MPS** **FUNCTION**

In [25]:
def create_initial_state_mps(x_grid):
  u_initial_vector = np.zeros(GRID_POINTS)
  u_initial_vector[x_grid <= 0.5] = 1.0
  # temp_tensor = tn.Node(u_initial_vector.reshape[2] * N_QUBITS)
  # mps_nodes =[]
  # current_node = temp_tensor
  # for i in range(N_QUBITS - 1):
  #   u, s, v, _ = tn.split_node_full_svd(
  #       current_node,
  #       left_edges=[current_node[0]],
  #       right_edges=[current_node[j] for j in range(1, len(current_node.edges))]
  #   )
  #   mps_nodes.append(u)
  #   current_node = tn.contract(tn.Node(np.diag(s)) @ v)
  # mps_nodes.append(current_node)
  # print(f"Successfully created initial state as an MPS with {len(mps_nodes)} nodes.")
  # return mps_nodes
  u_mps = tn.MatrixProductState.from_dense(
      u_initial_vector.reshape([2] * N_QUBITS),
      max_bond_dim=MAX_BOND_DIMENSION
  )
  return u_mps

**RUN SIMULATION FUNCTION**

In [26]:
def run_simulation():
  u_mps = create_initial_state_mps(x_grid)
  u_initial_vector = mps_to_vector(u_mps)
  for step in range(N_STEPS):
    # 1. Get the current vector as a classical vector
    u_vector = mps_to_vector(u_mps)
    # 2. Calculate derivatives classically
    du_dx_vector = np.gradient(u_vector, DX)
    d2u_dx2_vector = np.gradient(du_dx_vector, DX)
    # 3. Calculate the update term (the RHS of Burger's eqn)
    rhs_vector = -u_vector*du_dx_vector + VISCOSITY * d2u_dx2_vector
    # 4. Update the velocity vector
    u_next_vector = u_vector + DT * rhs_vector
    # 5. Convert the updated vector back to an MPS for the next step
    u_mps = tn.MatrixProductState.from_dense(
        u_next_vector.reshape([2] * N_QUBITS),
        max_bond_dim=MAX_BOND_DIMENSION
    )
    if (step + 1) % 20 == 0:
      print(f"Completed Step {step + 1} / {N_STEPS}")
  print(f"Simulation loop finished")
  final_node = mps_to_vector(u_mps)
  u_final_vector = final_node.tensor.flatten()

  # Plotting
  plt.figure(figsize=(12, 7))
  plt.plot(x_grid, u_initial_vector, 'bo-', label=f'Initial State (t=0)')
  plt.plot(x_grid, u_final_vector, 'ro-', label=f'Final State (t={N_STEPS*DT:.2f}s)')
  plt.title("1D Burgers' Equation using Tensor Network Method")
  plt.xlabel("Position (x)")
  plt.ylabel("Velocity (u)")
  plt.grid(True)
  plt.legend()
  plt.ylim(-0.1, 1.2)
  plt.show()

**MPS TO VECTOR FUNCTION**

In [10]:
def mps_to_vector(mps):
  return mps.to_dense().flatten()

**CREATE DERIVATION MPOS**

In [ ]:
def create_derivation_mpos(n_qubits, dx):
  d_dx_mpo = []
  for i in range(n_qubits):
    op = np.array([[0, 1], [-1, 0]]) / (2 * dx)
    mpo_tensor = np.zeroes((2, 2, 2, 2))
    mpo_tensor[0, 0, :, :] = np.eye(2)
    mpo_tensor[0, 1, :, :] = op
    mpo_tensor[1, 1, :, :] = np.eye(2)

    if i==0:
      d_dx_mpo.append(tn.Node(mpo_tensor[0, :, :, :]))
    elif i == n_qubits - 1:
      d_dx_mpo.append(tn.Node(mpo_tensor[:, 1, :, :]))
    else:
      d_dx_mpo.append(tn.Node(mpo_tensor))
  d2_d2x_mpo = []
  for i in range(n_qubits):
    op = np.array([[1, -2, 1], [0, 0, 0], [0, 0, 0]])
    op = op / (dx**2)
    mpo_tensor = np.zeroes((3, 3, 2, 2))
    mpo_tensor[0, 0, :, :] = np.eye(2)
    mpo_tensor[0, 1, :, :] = np.array([[1, 0], [0, 0]])
    mpo_tensor[0, 2, :, :] = np.array([[-2, 0], [0, 0]])
    mpo_tensor[1, 1, :, :] = np.eye(2)
    mpo_tensor[2, 1, :, :] = np.array([[1, 0], [0, 0]])
    mpo_tensor[2, 2, :, :] = np.eye(2)

    if i == 0:
      d2_d2x_mpo.append(tn.Node(mpo_tensor[0, :, :, :]))
    elif i == n_qubits - 1:
      d2_d2x_mpo.append(tn.Node(mpo_tensor[:, 2, :, :]))
    else:
      d2_d2x_mpo.append(tn.Node(mpo_tensor))
  return "d_dx_placeholder", "d2_dx2_placeholder"


In [27]:
if __name__ == "__main__":
  run_simulation()

AttributeError: module 'tensornetwork' has no attribute 'MatrixProductState'